In [1]:
import binascii
import sys
import traceback
from block import Block
from helper import read_varint, hash256
from tx import Tx
import subprocess
import os
from time import sleep
from subprocess import Popen, PIPE
import logging
import concurrent.futures
from multiprocessing import Pool
#logging.basicConfig(filename='parsing.log',level=logging.DEBUG,format='%(asctime)s %(message)s')

In [2]:
from neo4j import GraphDatabase

In [3]:
from logging.handlers import RotatingFileHandler

log_formatter = logging.Formatter('%(asctime)s %(levelname)s %(funcName)s(%(lineno)d) %(message)s')

logFile = 'log/parsing.log'

my_handler = RotatingFileHandler(logFile, mode='a', maxBytes=10*1024*1024,backupCount=6, encoding=None, delay=0)
my_handler.setFormatter(log_formatter)
my_handler.setLevel(logging.INFO)

app_log = logging.getLogger('root')
app_log.setLevel(logging.INFO)

app_log.addHandler(my_handler)

In [4]:
class BlockChainDB(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)#REVISE LATER FOR ENCRYPTION!!!

    def close(self):
        self._driver.close()
        
        
    @staticmethod
    def _new_address(tx, address,i,change_addr):
        if change_addr: addr_type = "change"
        else: addr_type = "recipient"
        result = tx.run("CREATE a = (:address {address:$address, acc_index:$index, type:$kind, created:timestamp()}) "
                        "RETURN a ", address=address, index = i, kind = addr_type)
        return result.single()
    
    
    @staticmethod
    def _new_tx(tx,block_id, version, locktime, tx_id, inputs, outputs, segwit, i):
                    
        result = tx.run("MATCH (b:block {id:$block_id}) "
                        "MERGE (t:transaction {id:$tx_id}) "
                        "SET t.version=$version, t.segwit=$segwit, t.locktime=$locktime "
                        "MERGE (t)<-[:CONTAINS {i:$i}]-(b) "
                        "RETURN t ",
                        tx_id=tx_id, segwit=segwit, version=version, locktime=locktime, block_id=block_id,i=i)
        
        if len(result.data()) >0: app_log.info(f"created tx: {tx_id}\n{result.data()}")
        else: app_log.error(f"FAILED AT CREATING TX {tx_id}")
        
        for index,tx_out in enumerate(outputs):
            result = tx.run("MATCH (t:transaction {id : $tx_id}) "
                        "MERGE (o:output {index:$index})<-[:CREATES]-(t) "
                        "SET o.amount=$amount, o.script_pubkey=$script_pubkey "
                        "RETURN o ", 
                        tx_id = tx_id,amount=tx_out.amount,script_pubkey=str(tx_out.script_pubkey), index=index)
            if len(result.data()) > 0: app_log.info(f"created output: {tx_out.amount}:{index} for tx {tx_id}")
            else: app_log.error(f"FAILED AT CREATING OUTPUT {index}")  
                
        for tx_in in inputs:
            if tx_in.prev_index == 4294967295 or tx_in.prev_tx == (b'\x00'*32).hex():
                    app_log.info("Coinbase Transaction " + tx_id)
            else: 
                result = tx.run("MERGE (t:transaction {id : $tx_id}) "
                            #"MERGE (p:transaction {id:$prev_tx})"
                            "MERGE (tx_in :output {index:$prev_index})<-[:CREATES]-(:transaction {id:$prev_tx}) "
                            "MERGE (tx_in)<-[r:SPENDS {script_sig:$script_sig, witness:$witness}]-(t) "
                            "RETURN r",
                            tx_id=tx_id, prev_index=tx_in.prev_index, prev_tx=tx_in.prev_tx.hex(), 
                            script_sig=str(tx_in.script_sig), witness=tx_in.witness.hex() )
                if len(result.data()) > 0: app_log.info(f"created input: {result.data()}")
                else: app_log.error(f"FAILED AT CREATING INPUT {tx_in.prev_tx}:{tx_in.prev_index}")  

        return True
    
    @staticmethod
    def _new_block(tx,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        #_height = tx.run( "MATCH (u:block) RETURN COUNT (u) ").single()[0] 
        #pblock = tx.run("MATCH (prev_block:block {id:$prev_block}) RETURN prev_block", prev_block=prev_block)
        #if not pblock.single():
         #   print(f"COULD NOT FIND PREVIOUS BLOCK {prev_block}.")
        result = tx.run("MERGE (block:block {id:$block_id}) "
                        "SET block.n_tx=$n_tx, block.nonce=$nonce, block.merkle_root=$merkle_root, block.bits=$bits, block.timestamp=$timestamp, block.version=$version "
                        "MERGE (prevblock:block {id:$prev_block}) "
                        "MERGE (block)<-[:chain]-(prevblock) "
                        "RETURN block ",
                        block_id=block_id, version=version, prev_block=prev_block, merkle_root=merkle_root, timestamp=timestamp, 
                        bits=bits, nonce=nonce, n_tx=n_tx)
        return result.data()

    def new_address(self, address,i,change_addr):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_address, address,i,change_addr)
            print(result)
            
    def new_tx(self, block_id, version, locktime, tx_id, inputs, outputs, segwit,i):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_tx, block_id, version, locktime, tx_id, inputs, outputs, segwit,i)
                  
    def new_block(self,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_block,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx)
            if len(result) >0: app_log.info(f"CREATED BLOCK {block_id}")
            else: app_log.error(f"FAILED AT CREATING BLOCK {block_id}")
            #print(result)
  
      

In [5]:
def get_cursor(file):
    c = 0
    print(f"checking for cursors {file}")
    try:
        cursor = open(f"cursors/cursor{file}.txt")
        c = cursor.readline()
        try:
            c = int(c)
            print(f"cursor at {c}")
            coursor.close()
            return c
        except:
            if c == "finished":
                print(c)
                return True
            else:
                try: 
                    print("trying to recover file from backup.")
                    cursor = open(f"{file[:-4]}.txt.bck")
                    c = coursor.readline()
                    try:
                        c = int(c)
                        subprocess.call(f" cp cursors/cursor{file}.txt.bck cursors/cursor{file}.txt", shell=True)
                        print("succesfully recovered file from backup. Restored original file.")
                        coursor.close()
                        return c
                    except:
                        if c == "finished":
                            coursor.close()
                            print(c)
                            subprocess.call(f" cp cursors/cursor{file}.txt.bck cursors/cursor{file}.txt",shell=True)
                            print("Finished. Succesfully updated original file.")
                            return c
                        else:
                            print("Corrupted cursor files.")
                            raise Exception

                except:
                    print("No back-up file.")
                    raise Exception
       

    except:
        cursor  = open(f"cursors/cursor{file}.txt","w")
        print("No cursor file")
        cursor.write(str(c))
        print(c)
        cursor.close()
        return c

def parse_blockchain(args):
    file = f"blocks_demo/blk{args[0]}.dat"
    db=args[1]
    print(f"parsing {file}")
    #print(f"driver {db}")
    
    with open(file,"rb") as block_file:
        print(f"opened {file}")
        c = get_cursor(args[0])
        if c !=0: 
            print(f"reading from {c}")
            block_file.read(c)
        
        while True:

            try:
                this_block = Block.parse_from_blk(block_file)

                header = this_block.version.to_bytes(4,"little")+this_block.prev_block[::-1]+this_block.merkle_root[::-1]+this_block.timestamp.to_bytes(4,"little") + this_block.bits + this_block.nonce
                block_id = hash256(header)[::-1]

                db.new_block(block_id.hex(),this_block.version, this_block.prev_block.hex(), 
                             this_block.merkle_root.hex(),this_block.timestamp, 
                             int.from_bytes(this_block.bits,"big"), 
                             int.from_bytes(this_block.nonce,"big"),
                             this_block.tx_hashes)

                for transaction in range(this_block.tx_hashes):
                    tx = Tx.parse(block_file)
                    db.new_tx(block_id.hex(), tx.version, tx.locktime, tx.id(), tx.tx_ins, tx.tx_outs, tx.segwit,transaction)

                c += (this_block.size + 8)
                #print(c)
                cursor  = open(f"cursors/cursor{args[0]}.txt","w")
                cursor.write(str(c))
                cursor.close()
                cursor  = open(f"cursors/cursor{args[0]}.txt.bck","w")
                cursor.write(str(c))
                cursor.close()
                
                
            except Exception as e:
                print(f"Finished {file} file. ")
                print(e)
                print(e.with_traceback)
                break
      

    
 

In [8]:
def main(args):
    n_threads = args[1]
    n = args[0]
    db = BlockChainDB( "neo4j://localhost:7687" ,"neo4j" ,"wallet" )
    file_list = [(f"{i:05}",db) for i in range( n , n + n_threads )]
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_threads) as executor:
        executor.map(parse_blockchain, file_list)   
    return True
        
def super_main(start_at,n_processes,n_threads):
    batch_list = [start_at+n_threads*i for i in range(n_processes)]
    arg_list = [(n,n_threads) for n in batch_list]
    with Pool(n_processes) as p:
        p.map(main, arg_list)
    
def manager(total_files, n_processes, n_threads, n_nodes):
    concurrent_files = n_processes*n_threads
    batches = total_files//concurrent_files+1
    batch_list = [concurrent_files*i for i in range(batches)]
    print(f"batch_list {batch_list}")
    for index,batch in enumerate(batch_list):
        node = index%n_nodes
        print(f"node {node} batch {batch}")
        super_main(batch, n_processes, n_threads)

In [ ]:
manager(10, 4,2,1)

batch_list [0, 8]
node 0 batch 0
parsing blocks_demo/blk00004.dat
parsing blocks_demo/blk00000.dat
parsing blocks_demo/blk00002.dat
parsing blocks_demo/blk00001.dat
parsing blocks_demo/blk00006.dat
parsing blocks_demo/blk00005.dat
parsing blocks_demo/blk00003.dat
parsing blocks_demo/blk00007.dat
opened blocks_demo/blk00006.dat
opened blocks_demo/blk00005.dat
opened blocks_demo/blk00004.dat
opened blocks_demo/blk00000.dat
opened blocks_demo/blk00001.dat
opened blocks_demo/blk00007.dat
opened blocks_demo/blk00002.dat
checking for cursors 00006
checking for cursors 00004
checking for cursors 00000
checking for cursors 00005
opened blocks_demo/blk00003.dat
checking for cursors 00001
checking for cursors 00002
checking for cursors 00007
cursor at 43942
checking for cursors 00003
cursor at 66491
cursor at 27391457
cursor at 25418670
cursor at 75195
cursor at 24567888
trying to recover file from backup.
trying to recover file from backup.
trying to recover file from backup.
cursor at 70919
cu

In [ ]:
main()

parsing blocks_demo/blk00000.dat
parsing blocks_demo/blk00001.dat
opened blocks_demo/blk00000.dat
checking for cursors 00000
parsing blocks_demo/blk00002.dat
opened blocks_demo/blk00001.dat
checking for cursors 00001
No cursor file
0
No cursor fileopened blocks_demo/blk00002.dat
checking for cursors 00002

0
No cursor file
0


In [32]:
os.chdir("/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin")
whereami = subprocess.check_output("pwd")
print(whereami)
if whereami == b"/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin\n": print("ok")
print(subprocess.check_output("ls"))
start_neo4j = subprocess.check_output("./neo4j start",shell = True)
if "Started neo4j" in str(start_neo4j): print("Sarted Neo4j successfully...")
else: print("Failed starting Neo4j.")
sleep(2)

In [57]:
#os.chdir("/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin")
#output = Popen(["./cypher-shell","-u","neo4j","-p","wallet"],stdout=PIPE, stderr=PIPE)
subprocess.call("./cypher-shell -u neo4j -p wallet",shell = True)
output = subprocess.check_output("MATCH (n) RETURN n",shell = True)
print(output)

CalledProcessError: Command 'MATCH (n) RETURN n' returned non-zero exit status 2.

In [54]:
stdout, stderr = output.communicate("\tMATCH (n) RETURN n\n")
print (stdout)
#subprocess.call("neo4j",shell = True)
#sleep(2)
#loggedin = subprocess.check_output("wallet",shell = True)
#if "Connected to Neo4j 4.0.3 at neo4j://localhost:7687 as user neo4j" in str(loggedin):print("connected")

b''


In [13]:
with open("blocks_demo/blk00000.dat","rb") as block_file:
    
    print(block_file.read(297))
    

b"\xf9\xbe\xb4\xd9\x1d\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00;\xa3\xed\xfdz{\x12\xb2z\xc7,>gv\x8fa\x7f\xc8\x1b\xc3\x88\x8aQ2:\x9f\xb8\xaaK\x1e^J)\xab_I\xff\xff\x00\x1d\x1d\xac+|\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xffM\x04\xff\xff\x00\x1d\x01\x04EThe Times 03/Jan/2009 Chancellor on brink of second bailout for banks\xff\xff\xff\xff\x01\x00\xf2\x05*\x01\x00\x00\x00CA\x04g\x8a\xfd\xb0\xfeUH'\x19g\xf1\xa6q0\xb7\x10\\\xd6\xa8(\xe09\t\xa6yb\xe0\xea\x1fa\xde\xb6I\xf6\xbc?L\xef8\xc4\xf3U\x04\xe5\x1e\xc1\x12\xde\\8M\xf7\xba\x0b\x8dW\x8aLp+k\xf1\x1d_\xac\x00\x00\x00\x00\xf9\xbe\xb4\xd9"


In [34]:
hex(20)

'0x14'

In [ ]:
def write_block():
    sleep(5)
    print("wrote block")
    
def write_tx():
    sleep(15)
    print("wrote transaction")

def parse_blockchain_demo(file):
    #db = BlockChainDB( "neo4j://localhost:7687" ,"neo4j" ,"wallet" )
    ok = True
    i = 0
    
    #while True:
        
    try:
        #with open(f"blocks_demo/blk{i:05}.dat","rb") as block_file:
        print(f"openning {file}")

        while ok:

            try:

                #read_file_and_parse_block
                sleep(5)
                print("parsed block")

                write_block()

                for transaction in range(random.randint(1, 2000)):
                    #read_file_and_parse_transaction
                    sleep(5)
                    print("parsed transaction")

                    write_tx()
                i+=1
                if i>=25: raise Exception

            except Exception as e:
                print(f"Finished blk{i:05} file. ")
                i+=1
                print(e)
                print(e.with_traceback)
                break
    except:
        print("Finsihed.")
        break

    
def main():
    file_list = [f"blocks_demo/blk{i:05}.dat" for i in range(2)]
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.map(parse_blockchain_demo, file_list)
        
main()

In [8]:
for r in p:
    print(r)

{'blk.id': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f', 'blk.prev_block': '0000000000000000000000000000000000000000000000000000000000000000'}
{'blk.id': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 'blk.prev_block': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'}
{'blk.id': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd', 'blk.prev_block': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048'}
{'blk.id': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449', 'blk.prev_block': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd'}
{'blk.id': '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485', 'blk.prev_block': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449'}
{'blk.id': '000000009b7262315dbf071787ad3656097b892abffd1f95a1a022f896f533fc', 'blk.prev_block': '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485'}
{'blk.id':

In [ ]:
os.chdir("block_demo")
files = subprocess.check_output("ls")
files